# Bidisperse Colloidal Ice

In this notebook we will simulate a Bidisperse Colloidal Ice. The parameters that we will use are the ones theoretically calculated to obtein a full degenerate Colloidal Spin Ice.

    r1 = 1 um
    susceptibility1 = 0.5
    trap_sep1 = 23 um
    
and
    
    r2 = 1 um
    susceptibility2 = 0.0675
    trap_sep2 = 30.3558 um
    
    
On the other hand the values of the field will be 25 mT and the total time 900 s. 

In [1]:
import sys
import os
import shutil
sys.path.insert(0, 'icenumerics/')

import pandas as pd
import numpy as np
import scipy.spatial as spa
import matplotlib.pyplot as plt
import matplotlib as mpl

import icenumerics as ice
from icenumerics.geometry import ordering 
import csv as csv
import time
import string as st
from multiprocessing import Pool
from string import Template

import copy as cp

ureg = ice.ureg

idx = pd.IndexSlice

import tqdm.auto as tqdm

%reload_ext autoreload
%autoreload 2

In [2]:
directory = "/home/carolina/Output_Sim_Summer_2020"
directory_DataFrame = "/home/carolina/DataFrames_Summer_2020"

# Design of the program for parallel preocesses

In [3]:
def do_everything(exp_entry):
    e = exp_entry[1].e 
    l = exp_entry[1].l
    
    
    # <To change the seed of the thermal noise>
    
    np.random.seed()
    
    # <Introduce the parameters for the simulation>

    lattice_constant = 33*ureg.um
    lattic_size = [l,l]
    sp = ice.spins()
    sp.create_lattice("square",[l,l],lattice_constant=33*ureg.um, border="periodic")
    sp.order_spins(ordering.random_ordering)

    particle1 = ice.particle(radius = 1*ureg.um,
             susceptibility = 0.5,
             diffusion = 0.125*ureg.um**2/ureg.s,
             temperature = 300*ureg.K,
             density = 1000*ureg.kg/ureg.m**3)

    trap1 = ice.trap(trap_sep = 23*ureg.um,
                   height = 0.5*ureg.pN*ureg.nm,
                   stiffness = 6e-4*ureg.pN/ureg.nm)

    particle2 = ice.particle(radius = 1*ureg.um,
             susceptibility = 0.0675,
             diffusion = 0.125*ureg.um**2/ureg.s,
             temperature = 300*ureg.K,
             density = 1000*ureg.kg/ureg.m**3)

    trap2 = ice.trap(trap_sep = 30.3558*ureg.um,
                   height = 0.5*ureg.pN*ureg.nm,
                   stiffness = 6e-4*ureg.pN/ureg.nm)

    # <In this loop we are generating our bidisperse Ice>
    
    traps = []
    particles = []

    for s in sp:

        if s.direction[1] == 0:
            # Horizontal traps
            traps.append(trap1) 
            particles.append(particle1)
            pass

        else: 
            # Vertical traps
            traps.append(trap2) 
            particles.append(particle2) 


    col = ice.colloidal_ice(sp, particles, traps , height_spread = 0.1 , susceptibility_spread = 0)

    
    # <Make the system periodic>
    
    col.region[:,:2]=(np.array([np.array([0,0]),lattic_size])-0.1)*lattice_constant
    col.region[:,2] = np.array([-.11,.11])*ureg.um
    
    # <Introduce the simulation parameters>
    
    world = ice.world(
    field = 200*ureg.mT,
    temperature = 300*ureg.K,
    dipole_cutoff = 200*ureg.um,
    boundaries = ["p", "p", "p"])

    change_m_time = 5800*ureg.s
    total_time = 11600*ureg.s
    col.simulation(world,
                 name = "BidisperseColloidalIce_Hysteresis-Loop_l%u_exp%u_"%(l,e),
                 include_timestamp = False,
                 targetdir = directory,
                 framerate = 1*ureg.Hz,
                 timestep = 10*ureg.ms,
                 run_time = total_time,
                 output = ["x","y","z","mux","muy","muz"])
    
    string_field = "(time<$Qt)*v_Bmag*time/%f+(time>$Qt)*v_Bmag*$Qt/$Qt-(time>$Qt)*v_Bmag*(time-$Qt)/$Qt"
    field_s = Template(string_field)
    field = field_s.substitute(Qt = 5800000000)
    col.sim.field.fieldz = field%(change_m_time.to(ureg.us).magnitude)
    col.run_simulation()
    
    # <Load simulation and compute vertices dataframes>
    
    col.load_simulation(slice(0,None,15))
    col.trj()
    v = ice.vertices()
    frames = col.trj.index.get_level_values("frame").unique()

    v_df = []

    for f in tqdm.tqdm(frames[::1]):
        col.set_state_from_frame(f)
        v = v.colloids_to_vertices(col)

        v_df.append(v.DataFrame())

    v_df = pd.concat(v_df, keys=frames[::1], names = ["frame"])
    
    # <Save the dataframe and create index of the runned simulation>
    
    v_df.to_csv(os.path.join(directory_DataFrame,"BidisperseColloidalIce_Hysteresis-Loop_l%u_exp%u"%(l,e)+".dat"), sep='\t')
    
    name = os.path.split(col.sim.base_name)[1]
    with open(os.path.join(directory,"index_Bidisperse.dat"),'a',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow([name, l, e])

In [4]:
e =  np.arange(0,1)
l =  np.arange(6,7)
L, E = np.meshgrid(l,e)
experiments = pd.DataFrame({"e":E.flatten(),"l":L.flatten()})

In [8]:
experiments.iterrows()

<generator object DataFrame.iterrows at 0x7f8626c709d0>

In [6]:
# %%time
if __name__ ==  '__main__': 
    num_processors = 6
    p=Pool(processes = num_processors)
    
    ## Create index text file
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(os.path.join(directory,"index_Bidisperse.dat"),'w',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow(["filename", "l", "exp"])
        
    list(tqdm.tqdm(p.imap(do_everything, experiments.iterrows()), total=len(experiments)))

TypeError: 'DataFrame' object is not callable